# Extract per gene dataset

This generates per gene data-set and surveys how many SNPs in each generated dataset.

In [1]:
%revisions -s -n 10

Revision,Author,Date,Message
d4e8a8c,Gao Wang,2018-05-24,Update documentation


In [ ]:
[global]
parameter: cwd = path('~/Documents/GTExV8')
parameter: genotype_file = f"{cwd:a}/GTExV7.genotype.cis.h5"
parameter: expr_file = f"{cwd:a}/GTExV7.expression.h5"
parameter: cov_file = f"{cwd:a}/GTExV7.covariates.h5"
parameter: outdir = f'{cwd:a}/Toys'
# we can extract single or multiple tissue data
parameter: tissues = ['Thyroid']
# list of genes to extract data for
parameter: genes = []
# if list of genes is not provided we'll get from a fixed file `n` genes
parameter: n = 200

[extract]
if len(genes) == 0:
    genes = get_output(f'shuf {cwd:a}/all_genes.txt | sort -u | head -{n} ').strip().split('\n')
tissue_tables = 'c(' + ','.join([f'"/{item}"' for item in tissues]) + ')'
input: for_each = 'genes', concurrent = True
output: f"{outdir}/{tissues[0] if len(tissues) == 1 else 'Multi_Tissues'}.{path(_genes):b}.RDS"
R: expand = '${ }'
    load_genotype = function(genotype_file, geno_table) {
            geno = h5read(genotype_file, geno_table)
            gdata = data.frame(geno$block0_values)
            colnames(gdata) = geno$axis1
            rownames(gdata) = geno$axis0
            return(gdata)
    }
    load_expression = function(expr_file, expr_table, gene) {
            expr = h5read(expr_file, expr_table)
            edata = data.frame(expr$block0_values)
            colnames(edata) = tools::file_path_sans_ext(expr$axis1)
            edata = data.frame(edata[, gene])
            rownames(edata) = expr$axis0
            colnames(edata) = gene
            return(edata)
    }
    load_covariates = funtion(cov_file, cov_table) {
            covariate <- h5read(cov_file, cov_table)
            cdata = data.frame(covariate$block0_values)
            colnames(cdata) = covariate$axis1
            rownames(cdata) = covariate$axis0
            cdata = t(cdata)
            return(cdata)
    }
    load_data_single = function(genotype_file, expr_file, cov_file, geno_table, expr_table, cov_table) {
            library(rhdf5)
            gdata = load_genotype(genotype_file, geno_table)
            edata = load_expression(expr_file, expr_table, basename(geno_table))
            cdata = load_covariates(cov_file, cov_table)
            # reorder
            edata = edata[match(rownames(cdata), rownames(edata)),]
            gdata = gdata[rownames(cdata),]
            return(list(X = as.matrix(gdata), y = as.vector(edata), Z = as.matrix(cdata), chrom = strsplit(colnames(gdata)[1], '_')[[1]][1], 
                        pos = as.integer(do.call(rbind, strsplit(colnames(gdata), '_'))[,2])))
    }
    load_data_multi = function(genotype_file, expr_file, cov_file, geno_table, expr_tables, cov_tables) {
            library(rhdf5)
            gdata = load_genotype(genotype_file, geno_table)
            edata = lapply(1:length(expr_tables), function(i) load_expression(expr_file, expr_tables[[i]], basename(geno_table)))
            cdata = lapply(1:length(cov_tables), function(i) load_covariates(cov_file, cov_tables[[i]]))
            y_res = list()
            # reorder and remove covariates
            for (i in 1:length(edata)) {
                edata[[i]] = edata[[i]][match(rownames(cdata[[i]]), rownames(edata[[i]])),]
                Z = scale(cdata[[i]], center=TRUE, scale=FALSE)
                y_res[[i]] = .lm.fit(Z, y_res[[i]])$residuals
                rownames(y_res[[i]]) = rownames(edata[[i]])
            }
            # merge residuals into one matrix
            # need to ensure same rows 
            # are matched and unmatched entries are filled with NA
            y_res_mat = NULL
            # and match its order with genotype data
            gdata = gdata[rownames(y_res_mat),]
            return(list(X = as.matrix(gdata), y = edata, Z = cdata, y_res = y_res_mat,
                        chrom = strsplit(colnames(gdata)[1], '_')[[1]][1], 
                        pos = as.integer(do.call(rbind, strsplit(colnames(gdata), '_'))[,2])))
    }
    #
    geno_table = "${_genes}"
    expr_table = cov_table = ${tissue_tables}
    if (length(expr_table) == 1) {
        dat = load_data_single(genotype_file, expr_file, cov_file, geno_table, expr_table, cov_table)
    } else {
        dat = load_data_multi(genotype_file, expr_file, cov_file, geno_table, expr_table, cov_table)        
    }
    saveRDS(dat, ${_output:r})

To get an idea how many variants are there:

In [ ]:
[count_var]
input: glob.glob('~/Documents/GTExV8/Toys/*.RDS'), group_by = 1
output: '~/Documents/GTExV8/Toys/counts.txt'
R: expand = True, stdout = _output
  cat(paste0({_input:br}, ncol(readRDS({_input:r})$X),'\n'))

In [1]:
%cd ~/Documents/GTExV8/Toys

/home/gaow/Documents/GTExV8/Toys

In [2]:
dat = scan('counts.txt')
summary(dat)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   3022    6450    7226    7217    8033   11999 